In [ ]:
import fastbook
fastbook.setup_book()

In [ ]:
from fastbook import *
from fastai.vision.widgets import *

In [ ]:
!jupyter serverextension enable --sys-prefix voila

In [ ]:
human_types = ['man','woman','boy','girl']

In [ ]:
path = Path('images\\humans')

In [ ]:
fns = get_image_files(path)
#fns

In [ ]:
failed = verify_images(fns)  #apparently any openable file will be included, including textfiles
#failed

In [ ]:
if failed: failed.map(Path.unlink()) #only run if there is failed images

In [ ]:
humans = DataBlock(
    blocks =(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=34),
    get_y=parent_label,
    item_tfms=RandomResizedCrop(224,min_scale=0.5),
    batch_tfms=aug_transforms()
)

In [ ]:
dls = humans.dataloaders(path, bs=64)

In [ ]:
dls.valid.show_batch(max_n=4, nrows = 1) # use in paperspace

In [ ]:
dls.train.show_batch(max_n=4, nrows = 1, unique = True) # use in paperspace 

In [ ]:
learn = cnn_learner(dls,resnet18,metrics=error_rate)
learn.fine_tune(4)

In [ ]:
#use only in paperspace
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
#use only in paperspace
interp.plot_top_losses(5, nrows=1)

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
#hide
for idx in cleaner.delete(): cleaner.fns[idx].unlink()
for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)

In [ ]:
learn.predict("images/test_cases/humans/man_h.jpg")

In [ ]:
learn.dls.vocab

In [ ]:
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
out_pl.clear_output()
lbl_pred = widgets.Label()
btn_run = widgets.Button(description='Classify')

In [ ]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]: .04f}'

In [ ]:
btn_run.on_click(on_click_classify)

In [ ]:
VBox([widgets.Label('Select your human'),btn_upload, btn_run, out_pl, lbl_pred])